In [0]:
# プロジェクトルートをパスに追加
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 環境変数の設定（STAGE=local の場合 .env から自動読み込み）
import os

os.chdir(project_root)  # プロジェクトルートに移動

# Jupyter Notebook用にnest_asyncioを有効化（asyncio.run()の問題を回避）
import nest_asyncio

nest_asyncio.apply()

# ライブラリのインポート
from demo.adapter.library import PdfProcessorClient

## 1. クライアントの初期化

`PdfProcessorClient`を初期化します。環境変数は`.env`ファイルから自動読み込みされます。

In [0]:
# クライアントを初期化
client = PdfProcessorClient(
    deployment_name="gpt-5-mini",  # LLMデプロイメント名
    embedding_deployment_name="text-embedding-3-small",  # Embeddingデプロイメント名
    show_progress=True,  # 進捗表示
)
print("✓ クライアントを初期化しました")

## 2. PDF処理の実行

`PdfProcessorClient`を使用してPDFを処理し、JSONファイルに保存します。

In [0]:
# 設定
pdf_path = "dataset/pdfs/1406.2661.pdf"  # 処理対象のPDFファイル
output_dir = "output"  # 出力ディレクトリ

# 出力ディレクトリの作成
Path(output_dir).mkdir(exist_ok=True)
print(f"PDF: {pdf_path}")
print(f"出力先: {output_dir}")

### PDF処理とJSON保存

`process_and_save_to_json()`メソッドでPDFを処理し、結果をJSONファイルに保存します。

In [0]:
# PDFを処理してJSONに保存
original_paper, search_data, paper_file, search_data_file = (
    client.process_and_save_to_json(
        pdf_file_path=pdf_path,
        output_dir=output_dir,
    )
)

print("\n✓ 処理が完了しました")
print(f"  論文情報: {paper_file}")
print(f"  検索データ: {search_data_file}")

## 3. 結果の確認

処理結果を直接確認します。

In [0]:
# 元論文情報を確認
print(f"ID: {original_paper.id}")
print(f"Title: {original_paper.title}")
print(f"Authors: {', '.join(original_paper.authors)}")
print(f"Abstract: {original_paper.abstract[:200]}...")

### 検索データの確認

In [0]:
# 検索データを確認
print(f"Total chunks: {len(search_data)}")
print("\nFirst chunk:")
print(f"  ID: {search_data[0].id}")
print(f"  Summary: {search_data[0].chunk_summary}")
print(f"  Keywords: {', '.join(search_data[0].keywords)}")
print(f"  Vector dimension: {len(search_data[0].chunk_vector)}")

In [0]:
### 保存されたJSONファイルの確認

In [0]:
# 出力されたファイルを確認
output_path = Path(output_dir)
json_files = list(output_path.glob("*.json"))
print(f"出力ファイル ({len(json_files)}件):")
for f in sorted(json_files):
    print(f"  - {f.name}")

## 4. 複数PDFの一括処理

ディレクトリ内の複数PDFを一括処理する例です。

In [0]:
# PDFディレクトリ内のファイルを処理
pdf_dir = Path("dataset/pdfs")
pdf_files = list(pdf_dir.glob("*.pdf"))

print(f"処理対象のPDFファイル ({len(pdf_files)}件):")
for pdf in pdf_files:
    print(f"  - {pdf.name}")

### 一括処理の実行（オプション）

以下のセルを実行すると、全PDFを一括処理します。処理には時間がかかります。

In [0]:
# 注意: このセルは時間がかかります。実行前に確認してください。
EXECUTE_BATCH = False  # Trueに変更すると一括処理を実行

if EXECUTE_BATCH:
    for pdf in pdf_files:
        print(f"\n{'=' * 50}")
        print(f"Processing: {pdf.name}")
        print(f"{'=' * 50}")

        try:
            paper, data, _, _ = client.process_and_save_to_json(
                pdf_file_path=str(pdf),
                output_dir=output_dir,
            )
            print(f"✓ {paper.title}: {len(data)} chunks")
        except Exception as e:
            print(f"⚠️ Error processing {pdf.name}: {e}")
else:
    print(
        "一括処理はスキップされました。実行するには EXECUTE_BATCH = True に変更してください。"
    )

## 5. データの分析

生成されたチャンクを分析します。

In [0]:
# チャンクの長さ分布
import matplotlib.pyplot as plt

chunk_lengths = [len(datum.chunk_text) for datum in search_data]

plt.figure(figsize=(10, 5))
plt.hist(chunk_lengths, bins=30, edgecolor="black")
plt.xlabel("Chunk Length (characters)")
plt.ylabel("Frequency")
plt.title("Distribution of Chunk Lengths")
plt.show()

print(f"Average chunk length: {sum(chunk_lengths) / len(chunk_lengths):.0f} characters")

### キーワード数の分布

In [0]:
# キーワード数の分布
keyword_counts = [len(datum.keywords) for datum in search_data]

plt.figure(figsize=(10, 5))
plt.hist(
    keyword_counts,
    bins=range(0, max(keyword_counts) + 2),
    edgecolor="black",
    align="left",
)
plt.xlabel("Number of Keywords")
plt.ylabel("Frequency")
plt.title("Distribution of Keywords per Chunk")
plt.show()

print(f"Average keywords per chunk: {sum(keyword_counts) / len(keyword_counts):.1f}")

In [0]:
### 頻出キーワード

In [0]:
# 最も頻出するキーワードの上位10個
from collections import Counter

all_keywords = []
for datum in search_data:
    all_keywords.extend(datum.keywords)

keyword_freq = Counter(all_keywords)
print("Top 10 most frequent keywords:")
for keyword, count in keyword_freq.most_common(10):
    print(f"  {keyword}: {count}")